In [1]:
import re # 정규화 용
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from Data_preprocessing import data_split
import joblib
from sklearn.metrics import *

In [3]:
train = pd.read_csv('hate_train.csv')
test = pd.read_csv('hate_test.csv') 
data = pd.read_csv('hate_all.csv') 
train.drop('Unnamed: 0', axis = 1, inplace = True)
test.drop('Unnamed: 0', axis = 1, inplace = True)


test_x = test.drop('hate', axis = 1)
test_y = test['hate']

df_x = train.drop('hate', axis = 1)
df_y = train['hate']

In [5]:
# 쓸데없는 문자 없애는 코드

def apply_preprocessing(sentence):
    import re
    '''
    영어, 숫자, 한글(온전한 문자)을 제외한 나머지 제거
    '''
    result = ''.join(re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]').\
             findall(sentence)).strip()
    
    return result

In [ ]:
# 랜덤 포레스트 모델
from sklearn.ensemble import RandomForestClassifier

x = df_x.copy()
y = df_y.copy()

x['comments'] = x['comments'].apply(apply_preprocessing) # 전처리

x['comments'] = x['comments'].apply(data_split) # 전처리

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer = 'char', sublinear_tf = True, ngram_range = (1,3), max_features = 10000)

train_inputs = vectorizer.fit_transform(x['comments'])
train_labels = np.array(y)

x_train, x_val, y_train, y_val = train_test_split(train_inputs, train_labels, test_size = 0.2, random_state = 42)

rf = RandomForestClassifier(max_depth=28, min_samples_leaf=1, min_samples_split=2, n_estimators=80)

rf.fit(x_train, y_train)

y_pred = rf.predict(x_val)
print(classification_report(y_val, y_pred))

In [6]:
# XGB 모델
from xgboost import XGBClassifier

x = df_x.copy()
y = df_y.copy()

x['comments'] = x['comments'].apply(apply_preprocessing) # 전처리

x['comments'] = x['comments'].apply(data_split) # 전처리

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer = 'char', sublinear_tf = True, ngram_range = (1,3), max_features = 10000)

train_inputs = vectorizer.fit_transform(x['comments'])
train_labels = np.array(y)

x_train, x_val, y_train, y_val = train_test_split(train_inputs, train_labels, test_size = 0.2, random_state = 42)

xgb4 = XGBClassifier(max_depth=20, 
		          eval_metric='logloss',  # 경고 메시지 제거
	                      learning_rate=0.5, 
	                      use_label_encoder=False,
	                      random_state=2022)

xgb4.fit(x_train, y_train)

C:\Users\User\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.5, max_delta_step=0,
              max_depth=20, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=2022,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [7]:
y_pred = xgb4.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      2165
           1       0.91      0.90      0.90      2004

    accuracy                           0.91      4169
   macro avg       0.91      0.91      0.91      4169
weighted avg       0.91      0.91      0.91      4169



In [8]:
test_x['comments'] = test_x['comments'].apply(apply_preprocessing) # 전처리

test_x['comments'] = test_x['comments'].apply(data_split) # 전처리

test_x = vectorizer.transform(test_x['comments'])
test_y = np.array(test_y)

y_pred = xgb4.predict(test_x)
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      1779
           1       0.91      0.89      0.90      1900

    accuracy                           0.90      3679
   macro avg       0.90      0.90      0.90      3679
weighted avg       0.90      0.90      0.90      3679



In [ ]:
# 최종 XGB 모델 선정

In [14]:
import joblib

joblib.dump(xgb4, 'xgb.pkl')
joblib.dump(vectorizer, 'vector.pkl')

['vector.pkl']

In [12]:
model = joblib.load('xgb.pkl')
y_pred = model.predict(test_x)
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      1779
           1       0.91      0.89      0.90      1900

    accuracy                           0.90      3679
   macro avg       0.90      0.90      0.90      3679
weighted avg       0.90      0.90      0.90      3679

